In [3]:
from configparser import Interpolation
import os
import cv2
from tqdm import tqdm
#from demo.image_matting.colab.inference2 import run
import numpy as np
from collections import Counter
import argparse
import configparser
import matplotlib.pyplot as plt


In [2]:
img = cv2.imread('/home/jini1114/git/data/dataset/data/r_cut_주르르.mp4/Images/7526.jpg')